In [1]:
import pandas as pd                                            # see below for install instruction
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
from scipy.stats import multivariate_normal
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [2]:
wiki = pd.read_csv('people_wiki.csv').head(5000)

In [3]:
wiki

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...
5,<http://dbpedia.org/resource/Sam_Henderson>,Sam Henderson,sam henderson born october 18 1969 is an ameri...
6,<http://dbpedia.org/resource/Aaron_LaCrate>,Aaron LaCrate,aaron lacrate is an american music producer re...
7,<http://dbpedia.org/resource/Trevor_Ferguson>,Trevor Ferguson,trevor ferguson aka john farrow born 11 novemb...
8,<http://dbpedia.org/resource/Grant_Nelson>,Grant Nelson,grant nelson born 27 april 1971 in london also...
9,<http://dbpedia.org/resource/Cathy_Caruth>,Cathy Caruth,cathy caruth born 1955 is frank h t rhodes pro...


In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

tf_idf = load_sparse_csr('4_tf_idf.npz')

In [5]:
import json
with open('4_map_index_to_word.json') as index_word_file:
    map_index_to_word = json.load(index_word_file)

In [6]:
map_index_to_word

{u'sundayswadsworth': 37318,
 u'verplank': 46431,
 u'damfunk': 23253,
 u'anyer': 62437,
 u'sonja': 37922,
 u'degussa': 33295,
 u'heraldleader': 55678,
 u'vinalop': 38962,
 u'chameleons': 15539,
 u'spiders': 76384,
 u'bienniale': 54484,
 u'hanging': 85150,
 u'woody': 94338,
 u'suzana': 67596,
 u'comically': 57840,
 u'localized': 44264,
 u'yougoslavia': 7189,
 u'sprague': 16230,
 u'gteborg': 82999,
 u'chanthaburi': 17970,
 u'igual': 52391,
 u'bandmeyns': 10335,
 u'gaa': 89074,
 u'storiesin': 84637,
 u'canek': 27791,
 u'petrak': 44943,
 u'naturopathic': 42153,
 u'obscura': 34481,
 u'bratislava': 86932,
 u'heroesin': 23118,
 u'theoryhe': 74282,
 u'broward': 85834,
 u'screaming': 85122,
 u'petras': 70455,
 u'grueling': 74796,
 u'vibrational': 53741,
 u'kangema': 42859,
 u'dcduring': 33588,
 u'elgan': 57644,
 u'wednesday': 93707,
 u'nielsenfunk': 24347,
 u'crotch': 13410,
 u'elgar': 70186,
 u'598m': 20672,
 u'perigeepenguin': 51015,
 u'mcelhinnycolleen': 2399,
 u'loanstopshop': 6262,
 u'vint

In [7]:
map_index_to_word = pd.DataFrame(map_index_to_word.items(), columns=['word', 'index'])

In [8]:
map_index_to_word

,word,index
0,sundayswadsworth,37318
1,verplank,46431
2,damfunk,23253
3,anyer,62437
4,sonja,37922
5,degussa,33295
6,heraldleader,55678
7,vinalop,38962
8,chameleons,15539
9,spiders,76384


In [9]:
tf_idf = normalize(tf_idf)

In [10]:
def diag(array):
    n = len(array)
    return spdiags(array, 0, n, n)

def logpdf_diagonal_gaussian(x, mean, cov):
    '''
    Compute logpdf of a multivariate Gaussian distribution with diagonal covariance at a given point x.
    A multivariate Gaussian distribution with a diagonal covariance is equivalent
    to a collection of independent Gaussian random variables.

    x should be a sparse matrix. The logpdf will be computed for each row of x.
    mean and cov should be given as 1D numpy arrays
    mean[i] : mean of i-th variable
    cov[i] : variance of i-th variable'''

    n = x.shape[0]
    dim = x.shape[1]
    assert(dim == len(mean) and dim == len(cov))

    # multiply each i-th column of x by (1/(2*sigma_i)), where sigma_i is sqrt of variance of i-th variable.
    scaled_x = x.dot( diag(1./(2*np.sqrt(cov))) )
    # multiply each i-th entry of mean by (1/(2*sigma_i))
    scaled_mean = mean/(2*np.sqrt(cov))

    # sum of pairwise squared Eulidean distances gives SUM[(x_i - mean_i)^2/(2*sigma_i^2)]
    return -np.sum(np.log(np.sqrt(2*np.pi*cov))) - pairwise_distances(scaled_x, [scaled_mean], 'euclidean').flatten()**2

In [11]:
def log_sum_exp(x, axis):
    '''Compute the log of a sum of exponentials'''
    x_max = np.max(x, axis=axis)
    if axis == 1:
        return x_max + np.log( np.sum(np.exp(x-x_max[:,np.newaxis]), axis=1) )
    else:
        return x_max + np.log( np.sum(np.exp(x-x_max), axis=0) )

def EM_for_high_dimension(data, means, covs, weights, cov_smoothing=1e-5, maxiter=int(1e3), thresh=1e-4, verbose=False):
    # cov_smoothing: specifies the default variance assigned to absent features in a cluster.
    #                If we were to assign zero variances to absent features, we would be overconfient,
    #                as we hastily conclude that those featurese would NEVER appear in the cluster.
    #                We'd like to leave a little bit of possibility for absent features to show up later.
    n = data.shape[0]
    dim = data.shape[1]
    mu = deepcopy(means)
    Sigma = deepcopy(covs)
    K = len(mu)
    weights = np.array(weights)

    ll = None
    ll_trace = []

    for i in range(maxiter):
        # E-step: compute responsibilities
        logresp = np.zeros((n,K))
        for k in xrange(K):
            logresp[:,k] = np.log(weights[k]) + logpdf_diagonal_gaussian(data, mu[k], Sigma[k])
        ll_new = np.sum(log_sum_exp(logresp, axis=1))
        if verbose:
            print(ll_new)
        logresp -= np.vstack(log_sum_exp(logresp, axis=1))
        resp = np.exp(logresp)
        counts = np.sum(resp, axis=0)

        # M-step: update weights, means, covariances
        weights = counts / np.sum(counts)
        for k in range(K):
            mu[k] = (diag(resp[:,k]).dot(data)).sum(axis=0)/counts[k]
            mu[k] = mu[k].A1

            Sigma[k] = diag(resp[:,k]).dot( data.multiply(data)-2*data.dot(diag(mu[k])) ).sum(axis=0) \
                       + (mu[k]**2)*counts[k]
            Sigma[k] = Sigma[k].A1 / counts[k] + cov_smoothing*np.ones(dim)

        # check for convergence in log-likelihood
        ll_trace.append(ll_new)
        if ll is not None and (ll_new-ll) < thresh and ll_new > -np.inf:
            ll = ll_new
            break
        else:
            ll = ll_new

    out = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}

    return out

In [12]:
from sklearn.cluster import KMeans

np.random.seed(5)
num_clusters = 25

# Use scikit-learn's k-means to simplify workflow
kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1)
kmeans_model.fit(tf_idf)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

In [13]:
means

[array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.01954039e-04,   1.04447276e-04,   9.87192763e-05]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.98056588e-04,   1.43796792e-04,   8.95769954e-05]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.16092642e-04,   1.45670542e-04,   8.38163507e-05]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.13464464e-04,   1.01278436e-04,   9.25652957e-05]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.94720759e-04,   8.29483688e-05,   1.16312561e-04]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.20138346e-04,   9.94621358e-05,   9.23794738e-05]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.16754523e-04,   1.27676596e-04,   8.65020280e-05]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.6927

In [15]:
num_docs = tf_idf.shape[0]
weights = []
for i in xrange(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = len(cluster_assignment[cluster_assignment == i])
    w = float(num_assigned) / num_docs
    weights.append(w)

In [17]:
weights

[0.0512,
 0.0256,
 0.0318,
 0.0132,
 0.01,
 0.0212,
 0.0782,
 0.0282,
 0.1676,
 0.017,
 0.0196,
 0.0526,
 0.0266,
 0.047,
 0.048,
 0.083,
 0.0168,
 0.0256,
 0.0166,
 0.0508,
 0.0182,
 0.0338,
 0.014,
 0.0688,
 0.0346]

In [18]:
sum(weights)

1.0

In [19]:
covs = []
for i in xrange(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

In [20]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)
print out['loglik']

[3855847476.6997027, 4844053202.4563665, 4844053202.4563665]


In [40]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')

    num_clusters = len(means)
    for c in xrange(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c])[::-1]

        for i in sorted_word_ids[:5]:
            word = map_index_to_word[map_index_to_word['index'] == i]['word']
            print '{0: <12}{1:<10.2e}{2:10.2e}'.format(word, 
                                                       means[c][i],
                                                       covs[c][i])
        print '\n=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]===='


In [41]:
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
35365    minister
Name: word, dtype: object7.57e-02    7.42e-03
9571    election
Name: word, dtype: object5.89e-02    3.21e-03
55669    party
Name: word, dtype: object5.89e-02    2.61e-03
12737    liberal
Name: word, dtype: object2.93e-02    4.55e-03
78889    elected
Name: word, dtype: object2.91e-02    8.95e-04

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
82263    film
Name: word, dtype: object1.76e-01    6.07e-03
30008    films
Name: word, dtype: object5.50e-02    2.97e-03
90293    festival
Name: word, dtype: object4.66e-02    3.60e-03
74657    feature
Name: word, dtype: object3.69e-02    1.81e-03
45729    directed
Name: word, dtype: object3.39e-02    2.22e-03

=========================================

In [47]:
a = map_index_to_word[map_index_to_word['index'] == 0]

In [48]:
print a['word']

35450    conflating
Name: word, dtype: object


In [49]:
np.random.seed(5)
num_clusters = len(means)
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    # YOUR CODE HERE
    mean = np.random.normal(loc = 0.0, scale = 1.0, size = num_words)
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    # YOUR CODE HERE
    cov = np.random.uniform(low = 1.0, high = 5.0, size = num_words)
    # Initially give each cluster equal weight.
    # YOUR CODE HERE
    weight = 1.0 / num_clusters
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

In [50]:
out_random_init = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-5)
print out_random_init['loglik']

[3855847476.6997027, 2362779935.0100083]


In [51]:
print out_random_init['loglik'][-1] - out['loglik'][-1]

-2481273267.45


In [52]:
visualize_EM_clusters(tf_idf, out_random_init['means'], out_random_init['covs'], 
                      map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
35365    minister
Name: word, dtype: object7.47e-02    7.40e-03
9571    election
Name: word, dtype: object5.85e-02    3.23e-03
55669    party
Name: word, dtype: object5.85e-02    2.62e-03
78889    elected
Name: word, dtype: object2.89e-02    9.13e-04
12737    liberal
Name: word, dtype: object2.89e-02    4.50e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
82263    film
Name: word, dtype: object1.70e-01    6.80e-03
30008    films
Name: word, dtype: object5.33e-02    2.98e-03
90293    festival
Name: word, dtype: object4.53e-02    3.55e-03
74657    feature
Name: word, dtype: object3.58e-02    1.80e-03
45729    directed
Name: word, dtype: object3.29e-02    2.20e-03

=========================================